In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
df = pd.read_csv('homework_1.1.csv')
df.head()

,Unnamed: 0,X1,X2,X3,Y
0,0,-0.440646,-0.390227,0.156718,-0.877671
1,1,-3.810099,-1.304665,-1.105117,-10.130388
2,2,-1.425451,-0.340049,1.115908,0.284068
3,3,-1.325750,0.161906,-0.254670,-1.994344
4,4,3.120263,1.487343,-1.164839,2.030030


In [5]:
X = df[['X1', 'X2', 'X3']]
y = df['Y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
coefficients = dict(zip(X.columns, model.coef_))
intercept = model.intercept_

mse, r2, coefficients, intercept

(0.27038744076297094,
 0.9908211302615575,
 {'X1': np.float64(1.0092442544468478),
  'X2': np.float64(1.9532067277273115),
  'X3': np.float64(2.960663144315282)},
 np.float64(0.00048148172336404116))

In [6]:
import statsmodels.api as sm

# Add a constant term for the intercept
X_with_const = sm.add_constant(X)

# Fit the OLS model
ols_model = sm.OLS(y, X_with_const).fit()

# Get the summary of the regression results
ols_summary = ols_model.summary()
ols_model.tvalues.sort_values(ascending=False)

X3       196.645240
X1        60.984011
X2        53.283212
const      0.166181
dtype: float64

In [7]:
simple_models = {
    col: sm.OLS(y, sm.add_constant(X[[col]])).fit()
    for col in X.columns
}

# Extract coefficients from simple regressions
simple_coefs = {col: model.params[col] for col, model in simple_models.items()}

# Extract coefficients from the multiple regression (already computed)
multi_coefs = ols_model.params.drop('const')

# Compute the absolute differences between the simple and multiple regression coefficients
coef_diff = {col: abs(simple_coefs[col] - multi_coefs[col]) for col in X.columns}

coef_diff

{'X1': np.float64(0.8346234440931537),
 'X2': np.float64(2.1190439844996054),
 'X3': np.float64(0.12155266688640776)}

In [10]:
df = pd.read_csv('homework_1.2.csv')

# Split the data into treatment and control groups based on X
treated = df[df['X'] == 1]
control = df[df['X'] == 0]

# Use Z variables for matching
Z_cols = [col for col in df.columns if col.startswith('Z')]
Z_treated = treated[Z_cols]
Z_control = control[Z_cols]

# Fit NearestNeighbors on the control group
nn = NearestNeighbors(n_neighbors=1)
nn.fit(Z_control)

# Find nearest neighbors for each treated unit
distances, indices = nn.kneighbors(Z_treated)
farthest_distance = distances.max()
print("Farthest match distance:", farthest_distance)

# Create a DataFrame of matched pairs
matched_control = control.iloc[indices.flatten()].copy()
matched_control.index = treated.index  # Align indices for easy comparison

# Optionally, concatenate matched pairs
matched_pairs = pd.concat([treated, matched_control], axis=1, keys=['Treated', 'Matched_Control'])

matched_pairs

Farthest match distance: 0.2102170871093757


Treated                        Matched_Control                       
   Unnamed: 0  X         Y         Z      Unnamed: 0  X         Y         Z
1           1  1  1.215189  0.715189              93  0  0.716327  0.716327
5           5  1  1.145894  0.645894              56  0  0.653108  0.653108
6           6  1  0.937587  0.437587              41  0  0.437032  0.437032
7           7  1  1.391773  0.891773              18  0  0.778157  0.778157
8           8  1  1.463663  0.963663              18  0  0.778157  0.778157
9           9  1  0.883442  0.383442              29  0  0.414662  0.414662
10         10  1  1.291725  0.791725              18  0  0.778157  0.778157
13         13  1  1.425597  0.925597              18  0  0.778157  0.778157
17         17  1  1.332620  0.832620              18  0  0.778157  0.778157
19         19  1  1.370012  0.870012              18  0  0.778157  0.778157
20         20  1  1.478618  0.978618              18  0  0.778157  0.778157
21         21  1  1.299159  0.799159              18  0  0.778157  0.778157
22         22  1  0.961479  0.461479              58  0  0.466311  0.466311
23         23  1  1.280529  0.780529              18  0  0.778157  0.778157
25         25  1  1.139921  0.639921              56  0  0.653108  0.653108
27         27  1  1.444669  0.944669              18  0  0.778157  0.778157
31         31  1  1.274234  0.774234              18  0  0.778157  0.778157
33         33  1  1.068434  0.568434              12  0  0.568045  0.568045
35         35  1  1.117635  0.617635              37  0  0.616934  0.616934
36         36  1  1.112096  0.612096              37  0  0.616934  0.616934
38         38  1  1.443748  0.943748              18  0  0.778157  0.778157
39         39  1  1.181820  0.681820              45  0  0.670638  0.670638
40         40  1  0.859508  0.359508              90  0  0.318569  0.318569
42         42  1  1.197631  0.697631              93  0  0.716327  0.716327
47         47  1  0.628926  0.128926              92  0  0.131798  0.131798
48         48  1  0.815428  0.315428              80  0  0.317983  0.317983
49         49  1  0.863711  0.363711              90  0  0.318569  0.318569
50         50  1  1.070197  0.570197              12  0  0.568045  0.568045
51         51  1  0.938602  0.438602              41  0  0.437032  0.437032
52         52  1  1.488374  0.988374              18  0  0.778157  0.778157
55         55  1  0.661310  0.161310              60  0  0.158970  0.158970
63         63  1  0.638183  0.138183              26  0  0.143353  0.143353
64         64  1  0.696582  0.196582              54  0  0.208877  0.208877
65         65  1  0.868725  0.368725              29  0  0.414662  0.414662
66         66  1  1.320993  0.820993              18  0  0.778157  0.778157
68         68  1  1.337945  0.837945              18  0  0.778157  0.778157
70         70  1  1.476459  0.976459              18  0  0.778157  0.778157
72         72  1  1.476761  0.976761              18  0  0.778157  0.778157
73         73  1  1.104846  0.604846               2  0  0.602763  0.602763
77         77  1  0.620197  0.120197              79  0  0.118728  0.118728
78         78  1  0.796140  0.296140              76  0  0.282807  0.282807
81         81  1  0.914263  0.414263              29  0  0.414662  0.414662
83         83  1  1.192472  0.692472              45  0  0.670638  0.670638
89         89  1  1.429296  0.929296              18  0  0.778157  0.778157
91         91  1  1.167410  0.667410              44  0  0.666767  0.666767
94         94  1  0.789406  0.289406              76  0  0.282807  0.282807
96         96  1  1.086513  0.586513              88  0  0.575946  0.575946
98         98  1  1.328940  0.828940              18  0  0.778157  0.778157

In [11]:
avg_y_treated = treated['Y'].mean()

# Compute the average Y for matched control units (X = 0 matched to each treated)
avg_y_matched_control = matched_control['Y'].mean()

# Calculate the treatment effect
effect = avg_y_treated - avg_y_matched_control
print("Estimated treatment effect:", effect)

Estimated treatment effect: 0.5433600651913855


In [14]:
# Fit NearestNeighbors with enough neighbors to check distances within threshold
nn_all = NearestNeighbors(radius=0.2)
nn_all.fit(Z_control)

# Find all neighbors within 0.2 radius for each treated observation
neighbors_within_radius = nn_all.radius_neighbors(Z_treated, return_distance=True)

# Extract indices and distances
distances_list, indices_list = neighbors_within_radius

# Build a list of matched pairs (treated index -> matched control indices)
matched_indices = []
for treated_idx, control_indices in zip(treated.index, indices_list):
    for control_idx in control_indices:
        matched_indices.append((treated_idx, control_idx))

# Create a DataFrame of matched control observations
matched_controls_within_radius = control.loc[[idx for _, idx in matched_indices]].copy()
matched_controls_within_radius['matched_to'] = [treated_idx for treated_idx, _ in matched_indices]

print(matched_controls_within_radius.head())

KeyError: '[1, 5, 9, 17, 20, 21, 25, 31, 35, 48, 27, 13, 39, 42, 22, 49, 6, 7, 8, 10, 19, 23, 33, 36, 38, 40, 47, 50, 51] not in index'